<a href="https://colab.research.google.com/github/trappertwo/rPPG-Toolbox/blob/main/%3CUpdated%3E_rPPG_on_compressed_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [1]:
# Clone the repository from github if not already setup
import os

dir_path = '/content/rPPG-Toolbox'
if os.path.isdir(dir_path):
  print(f"The directory '{dir_path}' exists.")
else:
  !git clone https://github.com/trappertwo/rPPG-Toolbox.git
%cd '{dir_path}'
!ls '{dir_path}'

Cloning into 'rPPG-Toolbox'...
remote: Enumerating objects: 3160, done.
remote: Counting objects: 100% (1324/1324), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 3160 (delta 1223), reused 1132 (delta 1132), pack-reused 1836 (from 2)
Receiving objects: 100% (3160/3160), 333.16 MiB | 31.98 MiB/s, done.
Resolving deltas: 100% (2180/2180), done.
Updating files: 100% (289/289), done.
/content/rPPG-Toolbox
 config.py	       LICENSE		  setup.sh
 configs	       main.py		  tools
 dataset	       model_outputs	  unsupervised_methods
 evaluation	       neural_methods	 '<Updated>_rPPG_on_compressed_videos.ipynb'
 figures	       README.md	  wip
 final_model_release   requirements.txt


In [2]:
# Explicitly install PyQt5 to use interactive plotting and avoid non-interactive backends
# See this relevant issue for more details: https://github.com/astral-sh/uv/issues/6893
# This requires Python 3.9+
!which python
!python --version
!pip install  --no-build-isolation PyQt5

/usr/local/bin/python
Python 3.11.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 26.0 MB/s eta 0:00:00


In [3]:
### Setup using uv
### This gives an error when building the mamba package.
### Hence we need to directly install causal-conv1d and mamba-ssm.
### See README file under tools/mamba.
#!bash setup.sh uv
%%bash
rm -rf .venv
uv venv --python 3.8
source .venv/bin/activate
uv pip install --no-build-isolation setuptools wheel
uv pip install --no-build-isolation torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 --index-url https://download.pytorch.org/whl/cu121
uv pip install --no-build-isolation -r requirements.txt
uv pip install --no-build-isolation causal-conv1d
uv pip install --no-build-isolation mamba-ssm


 Downloaded cpython-3.8.20-linux-x86_64-gnu
Using CPython 3.8.20
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
Resolved 2 packages in 117ms
 Downloaded setuptools
Prepared 2 packages in 162ms
Installed 2 packages in 87ms
 + setuptools==75.3.0
 + wheel==0.45.1
Resolved 19 packages in 1.16s
 Downloaded torchaudio
 Downloaded pillow
 Downloaded networkx
 Downloaded torchvision
 Downloaded numpy
 Downloaded sympy
 Downloaded triton
 Downloaded torch
Prepared 19 packages in 39.29s
Installed 19 packages in 472ms
 + certifi==2022.12.7
 + charset-normalizer==2.1.1
 + filelock==3.13.1
 + fsspec==2024.6.1
 + idna==3.4
 + jinja2==3.1.4
 + markupsafe==2.1.5
 + mpmath==1.3.0
 + networkx==3.3
 + numpy==1.24.1
 + pillow==10.2.0
 + requests==2.28.1
 + sympy==1.13.1
 + torch==2.1.2+cu121
 + torchaudio==2.1.2+cu121
 + torchvision==0.16.2+cu121
 + triton==2.1.0
 + typing-extensions==4.12.2
 + urllib3==1.26.13
Resolved 114 packages in 1.59s
   Building jax==0.4.6
   Build

In [4]:
### List packages in the virtual environment
!source .venv/bin/activate
!uv pip list

Package                      Version
---------------------------- ------------
absl-py                      2.1.0
asttokens                    3.0.0
astunparse                   1.6.3
backcall                     0.2.0
beautifulsoup4               4.13.3
cachetools                   5.5.2
causal-conv1d                1.0.0
certifi                      2022.12.7
charset-normalizer           2.1.1
comm                         0.2.2
cycler                       0.12.1
debugpy                      1.8.12
decorator                    5.1.1
einops                       0.8.1
executing                    2.2.0
filelock                     3.13.1
flatbuffers                  25.2.10
fsspec                       2024.10.0
gast                         0.4.0
gdown                        5.2.0
google-auth                  2.38.0
google-auth-oauthlib         1.0.0
google-pasta                 0.2.0
grpcio                       1.70.0
h5py                         2.10.0
huggingface-hub              

# PhysNet Training & Testing on Compressed Data


In [5]:
### UBFC-rPPG dataset in drive folder
### (TODO) Figure out how to import data set directly into colab
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Train on (compressed) UBFC-RPPG dataset using PhysNet

!source .venv/bin/activate && python main.py --config_file ./configs/train_configs/Compressed_UBFC-rPPG_UBFC-rPPG_UBFC-rPPG_PHYSNET_BASIC.yaml